**Table of contents**<a id='toc0_'></a>    
- [Importation des bibliothèques](#toc1_)    
  - [Importation des paquets ou modules de la bibliothèque OpenFL](#toc1_1_)    
  - [Importation des paquets ou modules de la bibliothèque PyTorch](#toc1_2_)    
  - [Importation d’autres paquets ou modules requis](#toc1_3_)    
- [Définition du modèle d‘entraînement](#toc2_)    
  - [Définition des chargeurs de données](#toc2_1_)    
  - [Définition du modèle de réseau CNN](#toc2_2_)    
  - [Définition de la fonction d'inférence utilisée dans le test](#toc2_3_)    
- [Définition des règles de l'apprentissage fédéré](#toc3_)    
  - [Méthode de calcul de la moyenne des poids d'apprentissage fédéré](#toc3_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Importation des bibliothèques](#toc0_)

## <a id='toc1_1_'></a>[Importation des paquets ou modules de la bibliothèque OpenFL](#toc0_)

In [1]:
# La classe `FLSpec` définit la spécification du flux.

# Les flux définis par l'utilisateur sont des sous-classes de cette classe.
from openfl.experimental.workflow.interface import Aggregator, Collaborator, FLSpec
# La fonction `aggregator/collaborator` est un décorateur de placement qui définit l'endroit où la 
# tâche sera assignée.
from openfl.experimental.workflow.placement import aggregator, collaborator
from openfl.experimental.workflow.runtime import LocalRuntime

## <a id='toc1_2_'></a>[Importation des paquets ou modules de la bibliothèque PyTorch](#toc0_)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchsummary import summary

from torchvision import datasets, transforms

## <a id='toc1_3_'></a>[Importation d’autres paquets ou modules requis](#toc0_)


In [3]:
from copy import deepcopy

import numpy as np

from termcolor import cprint

# <a id='toc2_'></a>[Définition du modèle d‘entraînement](#toc0_)

## <a id='toc2_1_'></a>[Définition des chargeurs de données](#toc0_)

In [4]:
random_seed = 1
torch.manual_seed(random_seed)

In [5]:
data_path = "/tmp/files/"

tensor_mnist = datasets.MNIST(
    data_path, train=True, download=True, transform=transforms.ToTensor()
)

tensor_images = torch.stack([tensor_image for tensor_image, _ in tensor_mnist], dim=3)

tensor_images.shape

torch.Size([1, 28, 28, 60000])

In [6]:
tensor_mean = tensor_images.view(1, -1).mean(dim=1)
tensor_mean

tensor([0.1307])

In [7]:
tensor_std = tensor_images.view(1, -1).std(dim=1)
tensor_std

tensor([0.3081])

In [8]:
"""
01. torchvision.transforms.Compose(transforms)
    - Composes several transforms together.

02. torchvision.transforms.Normalize(mean, std, inplace=False)
    - Normalize a tensor image with mean and standard deviation.
    - output[channel] = (input[channel] - mean[channel]) / std[channel]
"""

mnist_train = datasets.MNIST(
    "/tmp/files/",
    train=True,
    download=True,
    transform=transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(tensor_mean, tensor_std),
        ]
    ),
)

mnist_test = datasets.MNIST(
    "/tmp/files/",
    train=False,
    download=True,
    transform=transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(tensor_mean, tensor_std),
        ]
    ),
)

## <a id='toc2_2_'></a>[Définition du modèle de réseau CNN](#toc0_)

In [9]:
if torch.backends.mps.is_available():
    cprint("MPS is available", "green")
    device = torch.device("mps:0")
elif torch.backends.cuda.is_available():
    cprint("CUDA is available", "green")
    device = torch.device("cuda:0")
elif torch.backends.cudnn.is_built():
    cprint("CUDNN is available", "green")
    device = torch.device("cuda:0")
else:
    cprint("CUDA and MPS are not available", "red")
    cprint("Using CPU", "red")
    device = torch.device("cpu")

MPS is available


In [10]:
"""
03. torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1,
    groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
    - Applies a 2D convolution over an input signal composed of several input planes.

04. torch.nn.Dropout2d(p=0.5, inplace=False)
    - Randomly zero out entire channels.
    - Each channel will be zeroed out independently on every forward call with probability p using
    samples from a Bernoulli distribution.

05. torch.nn.functional.max_pool2d(input, kernel_size, stride=None, padding=0,
    dilation=1, ceil_mode=False, return_indices=False)
    - Applies a 2D max pooling over an input signal composed of several input planes.

06. torch.nn.functional.dropout(input, p=0.5, training=True, inplace=False)
    - During training, randomly zeroes some elements of the input tensor with probability p.
    - Uses samples from a Bernoulli distribution.

07. torch.nn.functional.log_softmax(input, dim=None, _stacklevel=3, dtype=None)
    - Apply a softmax followed by a logarithm.
    - While mathematically equivalent to log(softmax(x)), doing these two operations separately is
    slower and numerically unstable. This function uses an alternative formulation to compute the
    output and gradient correctly.
"""


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # La première couche convolutionnelle : le nombre de canaux d’entrée est de 1, c’est-à-dire 
        # une image en niveaux de gris, le nombre de canaux de sortie est de 10, la taille du filtre 
        # convolutif est de 5x5, le stride est de 1 et le padding est de 0.

        # Par conséquent, après que l'image d'entrée (1x28x28) a été convoluée, la taille de la 
        # carte de caractéristiques de sortie est de 10x24x24.
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        # La deuxième couche convolutionnelle : le nombre de canaux d’entrée est de 10, le nombre de 
        # canaux de sortie est de 20, la taille du filtre convolutif est de 5x5, le stride est de 1 
        # et le padding est de 0.

        # Par conséquent, après que l'image d'entrée (10x12x12) a été convoluée, la taille de la 
        # carte de caractéristiques de sortie est de 20x8x8.
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        # Une couche d'abandon.
        self.conv2_drop = nn.Dropout2d()
        # La première couche de fully connected : le nombre de canaux d’entrée est de 20, chaque 
        # canal a une taille de 4x4, soit un total de 20x4x4 = 320 nœuds, tandis que la sortie est 
        # fixée à 50 nœuds.
        self.fc1 = nn.Linear(320, 50)
        # La deuxième couche de fully connected : l'entrée a 50 nœuds et la sortie a 10 nœuds 
        # (correspondant à 10 catégories).
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        # La première couche convolutive est suivie d'une couche de pooling de type max pooling avec 
        # un filtre convolutif de taille de 2x2 et un stride égal à la longueur du filtre.

        # La taille d'entrée est de 10x24x24, et après pooling, la taille de sortie est de 10x12x12.
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        # La deuxième couche convolutive est suivie d'une couche d'abandon.

        # Après la couche d'abandon, suit une autre couche de max-pooling, qui possède un filtre 
        # convolutif de taille de 2x2 et un stride aussi égal à la longueur du filtre.

        # La taille d'entrée est de 20x8x8, et après pooling, la taille de sortie est de 20x4x4.
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        # La carte de caractéristiques multidimensionnelle est transformée en un vecteur 
        # unidimensionnel, d'une taille de 20x4x4 = 320.
        x = x.view(-1, 320)
        # La première couche de fully connected, activée par la fonction d'activation ReLU.
        x = F.relu(self.fc1(x))
        # Pendant l'entraînement du modèle, certains nœuds de la sortie de la première couche de 
        # fully connected sont mis à zéro de manière aléatoire avec une probabilité p.
        x = F.dropout(x, training=self.training)
        # La deuxième couche de fully connected sert également de couche de sortie.
        x = self.fc2(x)
        # Les probabilités logarithmiques de tous les nœuds de la couche de sortie sont calculées en 
        # appliquant une fonction softmax suivie d'un logarithme.
        return F.log_softmax(x, dim=1)

In [11]:
model = Net()
summary(model, next(iter(mnist_train))[0].shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 24, 24]             260
            Conv2d-2             [-1, 20, 8, 8]           5,020
         Dropout2d-3             [-1, 20, 8, 8]               0
            Linear-4                   [-1, 50]          16,050
            Linear-5                   [-1, 10]             510
Total params: 21,840
Trainable params: 21,840
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.08
Estimated Total Size (MB): 0.15
----------------------------------------------------------------


In [12]:
def count_parameters(model):
    length = 67
    names = [n for (n, p) in model.named_parameters() if p.requires_grad]
    name = "total parameters"
    names.append(name)
    max_length = max(map(len, names))
    formatted_names = [f"{f'  {n} ':.<{max_length + 3}}" for n in names]
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    params.append(sum(params))
    formatted_params = [f"{f' {p}  ':.>{length - max_length - 3}}" for p in params]

    for n, p in zip(formatted_names[:-1], formatted_params[:-1]):
        cprint((n + p), "magenta")
    cprint(" " + "_" * (length - 2) + " ", "magenta")
    cprint(
        (formatted_names[-1] + formatted_params[-1]),
        "magenta",
        end="\n\n",
    )

    return names, params


names, params = count_parameters(model)

  conv1.weight .............................................. 250  
  conv1.bias ................................................. 10  
  conv2.weight ............................................. 5000  
  conv2.bias ................................................. 20  
  fc1.weight .............................................. 16000  
  fc1.bias ................................................... 50  
  fc2.weight ................................................ 500  
  fc2.bias ................................................... 10  
 _________________________________________________________________ 
  total parameters ........................................ 21840  



## <a id='toc2_3_'></a>[Définition de la fonction d'inférence utilisée dans le test](#toc0_)

In [13]:
"""
08. torch.nn.functional.nll_loss(input, target, weight=None, size_average=None, ignore_index=-100,
    reduce=None, reduction='mean')
    - Compute the negative log likelihood loss.
"""


def inference(network, test_loader):
    # Mettre le module en mode évaluation.
    network.eval()
    network.to(device)
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = network(data)
            # L'entropie est une mesure de l'incertitude, c'est-à-dire que, si un résultat est 
            # certain, l'entropie est faible.

            # La perte d'entropie croisée, ou perte logarithmique, mesure les performances d'un 
            # modèle de classification dont le résultat est une valeur de probabilité comprise entre 
            # 0 et 1.

            # La perte d'entropie croisée augmente à mesure que la probabilité prédite s'écarte de 
            # l'étiquette réelle.

            # L’entropie croisée catégorielle sert au classement en plusieurs classes.

            test_loss += F.cross_entropy(output, target, reduction="sum").item()
            # Si `keepdim` est `True`, alors le tenseur de sortie est de la même taille que celui 
            # d'entrée, sauf dans la (les) dimension(s) `dim` où il est de dimension 1.
            pred = output.data.max(dim=1, keepdim=True)[1]
            # Calcul de l'égalité par éléments.
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    cprint(
        "Test set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        ),
        "magenta",
        attrs=["underline"],
        end="\n\n",
    )
    return float(correct / len(test_loader.dataset))

In [14]:
test_loader = DataLoader(mnist_test, batch_size=500, shuffle=False)

inference(model, test_loader)

Test set: Avg. loss: 2.3316, Accuracy: 1137/10000 (11%)



0.1137000024318695

# <a id='toc3_'></a>[Définition des règles de l'apprentissage fédéré](#toc0_)

## <a id='toc3_1_'></a>[Méthode de calcul de la moyenne des poids d'apprentissage fédéré](#toc0_)

In [15]:
def FedAvg(models, weights=None):
    new_model = models[0]
    state_dicts = [model.state_dict() for model in models]
    state_dict = new_model.state_dict()
    for key in models[1].state_dict():
        if state_dict[key].dim() != 0:
            state_dict[key] = torch.from_numpy(
                np.average(
                    [state[key].cpu().numpy() for state in state_dicts],
                    axis=0,
                    weights=weights,
                )
            )
        else:
            state_dict[key] = torch.from_numpy(
                np.average(
                    [state[key].reshape(1).cpu().numpy() for state in state_dicts],
                    axis=0,
                    weights=weights,
                )
            )
    new_model.load_state_dict(state_dict)
    return new_model

In [16]:
keys = [k for k in model.state_dict().keys()]
names = [n for (n, p) in model.named_parameters() if p.requires_grad]
keys == names

True

In [17]:
learning_rate = 0.01
log_interval = 10
momentum = 0.5

class FederatedFlow(FLSpec):

    def __init__(self, model=None, optimizer=None, rounds=3, **kwargs):
        super().__init__(**kwargs)
        # Importe un modèle personnalisé et ajoute le bon algorithme d’optimisation pour ce dernier.
        if model is not None:
            self.model = model
            self.optimizer = optimizer
        # Chargez le modèle `Net()` et configurez l'optimiseur pour qu'il s'applique uniquement à ce 
        # modèle.
        else:
            self.model = Net()
            self.optimizer = optim.SGD(
                self.model.parameters(), lr=learning_rate, momentum=momentum
            )
        self.rounds = rounds

    # Un agrégateur est le nœud central de l'apprentissage fédéré.

    # L'agrégateur commence par un modèle et un optimiseur transmis de manière facultative.

    # L'agrégateur commence le flux avec la tâche de `start`, où la liste des collaborateurs est 
    # extraite de l'exécution (`self.collaborators = self.runtime.collaborators`) et est ensuite 
    # utilisée comme liste de participants pour exécuter la tâche énumérée dans `self.next`, 
    # `aggregated_model_validation`.
    @aggregator
    def start(self):
        cprint("Performing initialization for model", "black", attrs=["bold"])
        self.collaborators = self.runtime.collaborators
        self.private = 10
        self.current_round = 0
        self.next(
            self.aggregated_model_validation,
            foreach="collaborators",
            exclude=["private"],
        )

    # Le modèle, l'optimiseur et tout ce qui n'est pas explicitement exclu de la fonction suivante 
    # seront transmis de la fonction de `start` de l'agrégateur à la tâche 
    # `aggregated_model_validation` du collaborateur.

    # L’endroit où les tâches sont exécutées est déterminé par le décorateur de placement qui 
    # précède chaque définition de tâche (`@aggregator` ou `@collaborator`).

    # Une fois que chaque collaborateur (défini dans l’exécution) a terminé la tâche 
    # `aggregated_model_validation`, il transmet son état actuel à la tâche `train`, de `train` à 
    # `local_model_validation`, et enfin à `join` à l'agrégateur.

    # C'est au niveau de `join` qu'une moyenne des poids des modèles est calculée et que le tour 
    # suivant peut commencer.
    @collaborator
    def aggregated_model_validation(self):
        cprint(
            f"Performing aggregated model validation for collaborator {self.input}",
            "red",
            attrs=["bold"],
        )
        self.agg_validation_score = inference(self.model, self.test_loader)
        cprint(
            f"{self.input} value of {self.agg_validation_score}",
            "red",
            attrs=["underline"],
        )
        self.next(self.train)

    @collaborator
    def train(self):
        self.model.train()
        self.optimizer = optim.SGD(
            self.model.parameters(), lr=learning_rate, momentum=momentum
        )
        for batch_idx, (data, target) in enumerate(self.train_loader):
            data, target = data.to(device), target.to(device)
            self.optimizer.zero_grad()
            output = self.model(data)
            loss = F.cross_entropy(output, target)
            loss.backward()
            self.optimizer.step()
            if batch_idx % log_interval == 0:
                cprint(
                    "Train Epoch: 1 [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                        batch_idx * len(data),
                        len(self.train_loader.dataset),
                        100.0 * batch_idx / len(self.train_loader),
                        loss.item(),
                    ),
                    "yellow",
                )
                self.loss = loss.item()
                torch.save(self.model.state_dict(), "model.pth")
                torch.save(self.optimizer.state_dict(), "optimizer.pth")
        self.training_completed = True
        self.next(self.local_model_validation)

    @collaborator
    def local_model_validation(self):
        self.local_validation_score = inference(self.model, self.test_loader)
        cprint(
            f"Doing local model validation for collaborator {self.input}: \
                {self.local_validation_score}",
            "white",
        )
        self.next(self.join, exclude=["training_completed"])

    @aggregator
    def join(self, inputs):
        self.average_loss = sum(input.loss for input in inputs) / len(inputs)
        self.aggregated_model_accuracy = sum(
            input.agg_validation_score for input in inputs
        ) / len(inputs)
        self.local_model_accuracy = sum(
            input.local_validation_score for input in inputs
        ) / len(inputs)
        cprint(
            f"Average aggregated model validation values = \
                {self.aggregated_model_accuracy}",
            "green",
        )
        cprint(f"Average training loss = {self.average_loss}", "green")
        cprint(
            f"Average local model validation values = \
            {self.local_model_accuracy}",
            "green",
        )
        self.model = FedAvg([input.model for input in inputs])
        self.optimizer = [input.optimizer for input in inputs][0]
        self.current_round += 1
        if self.current_round < self.rounds:
            self.next(
                self.aggregated_model_validation,
                foreach="collaborators",
                exclude=["private"],
            )
        else:
            self.next(self.end)

    @aggregator
    def end(self):
        cprint("This is the end of the flow", "black")

Aggregator step "start" registered
Collaborator step "aggregated_model_validation" registered
Collaborator step "train" registered
Collaborator step "local_model_validation" registered
Aggregator step "join" registered
Aggregator step "end" registered


In [18]:
batch_size_train = 64

# Configurer les participants.
aggregator = Aggregator()
aggregator.private_attributes = {}

# Configurer des collaborateurs avec des attributs privés.
collaborator_names = ["Portland", "Seattle", "Chandler", "Bangalore"]
collaborators = [Collaborator(name=name) for name in collaborator_names]
for idx, collaborator in enumerate(collaborators):
    local_train = deepcopy(mnist_train)
    local_test = deepcopy(mnist_test)
    local_train.data = mnist_train.data[idx :: len(collaborators)]
    local_train.targets = mnist_train.targets[idx :: len(collaborators)]
    local_test.data = mnist_test.data[idx :: len(collaborators)]
    local_test.targets = mnist_test.targets[idx :: len(collaborators)]
    collaborator.private_attributes = {
        "train_loader": DataLoader(
            local_train, batch_size=batch_size_train, shuffle=True
        ),
        "test_loader": DataLoader(
            local_test, batch_size=batch_size_train, shuffle=True
        ),
    }

# La classe `LocalRuntime` est un environnement d'exécution qui exécute le flux sur la machine
# locale.

# Il est utilisé pour le développement et le débogage.

# Il est également utilisé pour exécuter le flux sur un seul nœud.
local_runtime = LocalRuntime(
    aggregator=aggregator, collaborators=collaborators, backend="single_process"
)
print(f"Local runtime collaborators = {local_runtime.collaborators}")

Local runtime collaborators = ['Portland', 'Seattle', 'Chandler', 'Bangalore']


In [19]:
import os

if os.environ.get("USERNAME") is None:
    os.environ["USERNAME"] = "Hao"

In [20]:
import getpass
print(getpass.getuser())

haozhang


In [21]:
model = None
best_model = None
optimizer = None
flflow = FederatedFlow(model, optimizer, rounds=2, checkpoint=True)
flflow.runtime = local_runtime
flflow.run()

Created flow FederatedFlow

Calling start
Performing initialization for model
Saving data artifacts for start
Saved data artifacts for start

Calling aggregated_model_validation
Performing aggregated model validation for collaborator Portland
Test set: Avg. loss: 2.3249, Accuracy: 121/2500 (5%)

Portland value of 0.04839999973773956
Saving data artifacts for aggregated_model_validation
Saved data artifacts for aggregated_model_validation

Calling train
Train Epoch: 1 [0/15000 (0%)]	Loss: 2.347810
Train Epoch: 1 [640/15000 (4%)]	Loss: 2.308646
Train Epoch: 1 [1280/15000 (9%)]	Loss: 2.317288
Train Epoch: 1 [1920/15000 (13%)]	Loss: 2.239920
Train Epoch: 1 [2560/15000 (17%)]	Loss: 2.222916
Train Epoch: 1 [3200/15000 (21%)]	Loss: 2.155879
Train Epoch: 1 [3840/15000 (26%)]	Loss: 2.114819
Train Epoch: 1 [4480/15000 (30%)]	Loss: 1.977747
Train Epoch: 1 [5120/15000 (34%)]	Loss: 1.961372
Train Epoch: 1 [5760/15000 (38%)]	Loss: 1.792134
Train Epoch: 1 [6400/15000 (43%)]	Loss: 1.741473
Train Epoch

In [22]:
print(
    f'Sample of the final model weights: {flflow.model.state_dict()["conv1.weight"][0]}'
)

print(
    f"\nFinal aggregated model accuracy for {flflow.rounds} rounds of training: {flflow.aggregated_model_accuracy}"
)

Sample of the final model weights: tensor([[[ 0.1473,  0.0277, -0.2045, -0.0369, -0.1523],
         [-0.0349, -0.1079,  0.1950,  0.2622,  0.1505],
         [ 0.2971,  0.2129,  0.0031,  0.1970,  0.1009],
         [ 0.2575,  0.1428,  0.0088,  0.2224,  0.0102],
         [ 0.0856, -0.1701, -0.1939, -0.0512,  0.0848]]], device='mps:0')

Final aggregated model accuracy for 2 rounds of training: 0.8691000193357468
